# DSCI 100 Group Project Report

### Introduction

Can we predict the win rate of a player given their various statistics? We aim to investigate the factors that influence the win rate in tennis games. We are interested in understanding the relationship between a player's stats and the likelihood of winning a game. By gaining insights into how these factors relate to the probability of winning, we can make more informed and predictive decisions regarding future tennis matches.

To conduct our research, we chose to use the `atp2017-2019.csv` dataset provided on the group project proposal page on Canvas. This dataset contains 6866 observations and 49 variables. Each observation represents a match played in 2017-2019 between top 500 ATP ranked players. The variables contain information both about the match and the individual player stats, with 11 being unique variables, and the rest being various player stats that are separate for the winner and the loser.

### Methods & Results

The following libraries are loaded to provide the functions that are used to operate on the data:

In [47]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 10)

The data is read using `read_csv` and displayed below.

In [48]:
tennis_data <- read_csv("atp2017-2019.csv") |>
    select(-...1) |>
    mutate(surface = as_factor(surface))
tennis_data

New names:
• `` -> `...1`
Rows: 6866 Columns: 50
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (16): tourney_id, tourney_name, surface, tourney_level, winner_seed, win...
dbl (34): ...1, draw_size, tourney_date, match_num, winner_id, winner_ht, wi...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,⋯,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
<chr>,<chr>,<fct>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2,NA,⋯,54,34,20,14,10,15,9,3590,16,1977
2019-M020,Brisbane,Hard,32,A,20181231,299,106421,4,NA,⋯,52,36,7,10,10,13,16,1977,239,200
2019-M020,Brisbane,Hard,32,A,20181231,298,105453,2,NA,⋯,27,15,6,8,1,5,9,3590,40,1050
2019-M020,Brisbane,Hard,32,A,20181231,297,104542,NA,PR,⋯,60,38,9,11,4,6,239,200,31,1298
2019-M020,Brisbane,Hard,32,A,20181231,296,106421,4,NA,⋯,56,46,19,15,2,4,16,1977,18,1855
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2017-0605,Tour Finals,Hard,8,F,20171113,300,105777,6.0,NA,⋯,54,42,22,15,11,15,6,3650,8,2975
2017-M-DC-2017-WG-M-BEL-FRA-01,Davis Cup WG F: BEL vs FRA,Hard,4,D,20171124,1,105676,NA,NA,⋯,53,33,21,14,6,11,7,3775,18,2235
2017-M-DC-2017-WG-M-BEL-FRA-01,Davis Cup WG F: BEL vs FRA,Hard,4,D,20171124,2,104542,NA,NA,⋯,54,30,12,12,5,11,15,2320,76,667


The stats of the winners of each match are selected and renamed, then assigned to an object named *`tennis_winner`*. The number of wins is also obtained using `group_by()` and `summarize()` and assigned to *`tennis_wins`*.

In [68]:
tennis_winner <- tennis_data |>
    select(winner_name, winner_age, minutes:w_bpFaced) |>
    rename("name" = winner_name, "age" = winner_age, "ace" = w_ace, "df" = w_df, "svpt" = w_svpt, "1stIn" = w_1stIn, "1stWon" = w_1stWon,
           "2ndWon" = w_2ndWon, "SvGms" = w_SvGms, "bpSaved" = w_bpSaved, "bpFaced" = w_bpFaced)

tennis_wins <- tennis_winner |>
    group_by(name) |>
    summarize(winCount = n())

tennis_win_names <- tennis_winner |>
    pull(name)

The stats of the losers are also selected and renamed, but are then filtered to only contain those that have won at least a single match. The number of losses is obtained using the same operations.

In [75]:
tennis_loser <- tennis_data |>
    select(loser_name, loser_age, minutes, l_ace:l_bpFaced) |>
    rename("name" = loser_name, "age" = loser_age, "ace" = l_ace, "df" = l_df, "svpt" = l_svpt, "1stIn" = l_1stIn, "1stWon" = l_1stWon,
           "2ndWon" = l_2ndWon, "SvGms" = l_SvGms, "bpSaved" = l_bpSaved, "bpFaced" = l_bpFaced) |>
    filter(name %in% tennis_win_names)

tennis_losses <- tennis_loser |>
    group_by(name) |>
    summarize(lossCount = n())

tennis_loss_names <- tennis_loser |>
    pull(name)

The number of wins and losses are put into a single table here. `filter()` is used here again, this time to filter to only contain those that have lost at least a single 
match. This creates a better representation of players, as those that have only won or only lost matches over two years are likely outliers who haven't played many games.

In [72]:
win_loss <- tennis_wins |>
    filter(name %in% tennis_loss_names) |>
    bind_cols(select(tennis_losses, lossCount))

Finally, all the data gathered above is put together and assigned to *`tennis_stats`*. This dataframe contains all players with at least a single win and loss, including their average stats, number of games played, and numbers of wins and losses.

In [70]:
tennis_stats <- bind_rows(tennis_winner, tennis_loser) |>
    filter(name %in% tennis_loss_names) |>
    group_by(name) |>
    summarize(across(age:bpFaced, mean, na.rm = TRUE), gamesPlayed = n()) |>
    bind_cols(select(win_loss, winCount, lossCount))
tennis_stats

name,age,minutes,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced,gamesPlayed,winCount,lossCount
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
Adam Pavlasek,22.94593,97.50000,6.750000,3.250000,82.75000,51.62500,35.00000,13.62500,12.87500,5.000000,9.000000,8,3,5
Adrian Mannarino,29.97470,107.40764,4.848101,2.373418,78.07595,48.01266,33.35443,15.55696,12.20886,4.025316,6.778481,158,81,77
Adrian Menendez Maceiras,32.46032,109.13333,3.800000,2.133333,78.53333,48.53333,32.06667,14.20000,11.60000,4.066667,7.600000,15,4,11
Akira Santillan,20.06069,75.66667,6.166667,2.833333,65.33333,35.00000,25.83333,16.33333,10.66667,2.000000,4.000000,6,2,4
Albert Ramos,30.31832,120.15686,3.823529,2.483660,81.73203,50.62745,34.56863,16.21569,12.48366,4.477124,7.320261,153,72,81
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Yuichi Sugita,29.39904,106.7143,3.728571,2.785714,80.24286,49.00000,33.64286,15.44286,12.27143,4.385714,7.400000,70,29,41
Ze Zhang,27.82971,98.3000,5.300000,4.200000,76.40000,50.40000,35.70000,11.60000,11.70000,3.500000,6.200000,10,2,8
Zhe Li,32.72553,122.6667,2.666667,1.166667,82.83333,55.16667,33.50000,15.50000,12.66667,4.166667,8.500000,7,1,6


### Discussion

### References